## Python Design Patterns I (Chapter 10)

### Decorator Pattern

*allows us to "wrap" an object that provides core functionality with other objects that alter this functionality. Any object that uses the decorated object will interact with it in exactly the same way as if it were undecorated* (Dusty Phillips, pg 301)

*A function that takes another function as an argument and adds some kind of functionality and then returns another function. All of this without altering the source code of the original function that you passed in.* (Corey Schafer <a href="https://www.youtube.com/watch?v=FsAPt_9Bf3U" target="_blank">min 4:48</a>)

#### Why Use It?

* to expand on the response of a function/component, before/as the response is sent to a second component
* to allow for multiple optional behaviors (as an alternative to multiple inheritance)

In [1]:
# Quick review of nested functions
def outer_function():
    message = 'Hi!'
    
    def inner_function():
        print(message)
   
    # inner_function is called
    return inner_function()

In [2]:
# We can call the outer_function
outer_function()

Hi!


In [5]:
# But we cannot create a new instance of the outer_function
my_func = outer_function()
my_func()

Hi!


TypeError: 'NoneType' object is not callable

In [4]:
# Quick review of closures:
    # Requires nested function that refers to a value defined
    # in the enclosing function, which returns the nested function
def outer_function():
    message = 'Hi!'
    
    def inner_function():
        print(message)
    
    # Closure returns the inner function 
    # that can access variables created in local scope
    return inner_function

In [5]:
# inner_function is returned, waiting to be executed
outer_function()

<function __main__.outer_function.<locals>.inner_function()>

In [6]:
# Now, we can create a new instance of the outer_function
my_func = outer_function()
my_func()

Hi!


In [9]:
# Closures With Arguments
def outer_function(msg):
    def inner_function():
        print(msg)    
    return inner_function

In [10]:
# Closure returns the message variable
# passed to the outer_function
hi_func = outer_function('Hi!')
bye_func = outer_function('Bye!')

hi_func()
bye_func()

Hi!
Bye!


In [13]:
# From closure, which accepts a message as an argument
def decorator_function(msg):
    def wrapper_function():
        print(msg)    
    return wrapper_function

In [14]:
# To decorator, which accepts a function as an argument
def decorator_function(original_function):
    def wrapper_function():
        
        # So we can add functionality to the wrapper
        # without the need to modify the original_function
        print('wrapper executed this before {}'.format(original_function.__name__))        
        return original_function()  
    return wrapper_function

def display():
    print('display function ran')

In [15]:
# display function is passed to decorator_function
# which returns a wrapper function that is waiting to be executed
decorated_display = decorator_function(display)
decorated_display()

wrapper executed this before display
display function ran


In [16]:
# Cleaner implementation of decorator
def decorator_function(original_function):
    def wrapper_function():
        print('wrapper executed this before {}'.format(original_function.__name__))        
        return original_function()  
    return wrapper_function

# This syntax clearly indicates that function has been decorated at definition
@decorator_function
def display():
    print('display function ran')

In [17]:
# No need to explicitly pass original_function to decorator_function
# i.e. decorated_display = decorator_function(display)
display()

wrapper executed this before display
display function ran


In [18]:
# What if original_function takes arguments?
def decorator_function(original_function):
    def wrapper_function():
        print('wrapper executed this before {}'.format(original_function.__name__))        
        return original_function()  
    return wrapper_function

@decorator_function
def display_info(name, age):
    print('display_info function ran with arguments ({}, {})'.format(name, age))

In [19]:
# Does not work
display_info("Jane", 50)

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

In [20]:
# We need to be able to pass a variable number of arguments to wrapper
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print('wrapper executed this before {}'.format(original_function.__name__))        
        return original_function(*args, **kwargs)  
    return wrapper_function

@decorator_function
def display_info(name, age):
    print('display_info function ran with arguments ({}, {})'.format(name, age))

In [21]:
display_info("Jane", 50)

# For decorators that take arguments, see: https://www.youtube.com/watch?v=KlBPCzcQNU8&t=3s

wrapper executed this before display_info
display_info function ran with arguments (Jane, 50)


### Observer Pattern

*useful for state monitoring and event handling situations... allows a given object to be monitored by an unknown and dynamic group of "observer" objects* (Dusty Phillips, pg 307)

#### Why Use It?

* when value in core object changes, observer objects are notified using an update() method
* detaches the code being observed from the code doing the observing
    * core object doesn't need to know about observers
    * can be multiple observers responsible for different tasks after a change to the core object 
    * no need to include the core object's code within the observer code

In [39]:
# Observed object (also referred to as publisher)
class Inventory:
    def __init__(self):
        self.observers = []
        self._product = None
        self._quantity = 0
    
    # Attach observer to the inventory object
    # Also known as: attach subscriber to publisher
    def attach(self, observer):
        self.observers.append(observer)
    
    # Set product and update observers
    @property
    def product(self):
        return self._product
    @product.setter
    def product(self, value):
        self._product = value
        self._update_observers()
    
    # Set quantity and update observers
    @property
    def quantity(self):
        return self._quantity
    @quantity.setter
    def quantity(self, value):
        self._quantity = value
        self._update_observers()
    
    # Call observer to update
    def _update_observers(self):
        for observer in self.observers:
            observer()

In [40]:
# Simple observer object that prints info from observed
class ConsoleObserver:
    def __init__(self, inventory):
        self.inventory = inventory
        
    def __call__(self):
        print(self.inventory.product)
        print(self.inventory.quantity)

In [41]:
# Instance of observed (publisher)
i = Inventory()

# Instance of observer (subscriber)
c = ConsoleObserver(i)

# Attach observer to observed
i.attach(c)

# Changes to observed result in call to observer
i.product = "Widget"
i.quantity = 5

Widget
0
Widget
5


In [42]:
# There can be multiple instances of observer
i = Inventory()
c1 = ConsoleObserver(i)
c2 = ConsoleObserver(i)

i.attach(c1)
i.attach(c2)

i.product = "Gadget"

Gadget
0
Gadget
0


In [43]:
i.quantity = 2

Gadget
2
Gadget
2


### Strategy Pattern

*common demonstration of abstraction in object-oriented programming. The pattern implements different solutions to a single problem, each in a different object. The client code can then choose the most appropriate implementation dynamically at runtime* (Dusty Phillips, pg 310)

#### Why Use It?

* used to abstract the code doing the work from the code calling the function (i.e. Modularity)
* performs same task in different ways, but nothing changes for the end user/interface/client initiating the task

In [9]:
# Simple example: sorted()
a_list = [5, 2, 3, 1, 4]
a_dict = {1: 'D', 2: 'B', 3: 'B', 4: 'E', 5: 'A'}
tuples = [
    ('john', 'A', 15),
    ('jane', 'B', 12),
    ('dave', 'B', 10),
    ]

print(sorted(a_list))
print(sorted(a_dict))
print(sorted(tuples, key=lambda student: student[2]))  # sort by age

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]


In [12]:
# Example from book
from PIL import Image

# Each strategy has a make_background method with the same arguments

# Loops over input image and based on width/height copies it 
# into the appropriate tile location
class TiledStrategy:
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = Image.new('RGB', desktop_size)
        num_tiles = [
        o // i + 1 for o, i in
        zip(out_img.size, in_img.size)
        ]
        for x in range(num_tiles[0]):
            for y in range(num_tiles[1]):
                out_img.paste(
                    in_img,
                    (
                        in_img.size[0] * x,
                        in_img.size[1] * y,
                        in_img.size[0] * (x+1),
                        in_img.size[1] * (y+1)
                    )
                )
        return out_img

# Figures out how much space needs to be left on edges to center image
class CenteredStrategy:
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = Image.new('RGB', desktop_size)
        left = (out_img.size[0] - in_img.size[0]) // 2
        top = (out_img.size[1] - in_img.size[1]) // 2
        out_img.paste(
            in_img,
            (
                left,
                top,
                left+in_img.size[0],
                top + in_img.size[1]
            )
        )
        return out_img

# Sets the image to the output size (ignoring aspect ratio)
class ScaledStrategy:
    def make_background(self, img_file, desktop_size):
        in_img = Image.open(img_file)
        out_img = in_img.resize(desktop_size)
        return out_img

### Typical Python Approach

* previous approach not as common in Python, as other object-oriented languages
    * because each class only provides a single function, might as well just call function directly like sorted()
* typically implemented in Python using first-class functions, as they can be used like any other object:
    * used as parameters
    * used as a return value
    * assigned to variables...

### State Pattern

*represent state-transition systems... need a manager, or context class that provides an interface for switching states. Internally, this class contains a pointer to the current state; each state knows what other states it is allowed to be in and will transition to those states depending on actions invoked upon it* (Dusty Phillips, pg 313)

#### Why Use It?

* allow objects to switch between different states dynamically, as some process evolves
* looks similar to strategy pattern but strategy is used to choose appropriate code at runtime
    * instead state pattern relies on the states (or context) to know which other states it can switch it dynamically if needed  
* can be implemented in various ways in Python: first-class functions, context managers, coroutines

In [3]:
import rasterio as rio
import matplotlib.pyplot as plt

raster_path = '../n39_w106_3arc_v2.tif'

# When rasterio context manager is entered:
    # GDAL drivers are registered
    # error handlers are configured
    # configuration options are set
# When rasterio context manager is exited:
    # drivers are removed from the registry
    # other configurations are removed
with rio.open(raster_path) as src:
    image = src.read()

In [4]:
# Equivalent to 
src = rio.open(raster_path)
image = src.read()
src.close()

In [13]:
# Achieve similar effect to state pattern using coroutines
def grep(pattern):
    print("Searching for", pattern)
    while True:
        line = (yield) # values supplied externally
        if pattern in line:
            print(line)

In [14]:
search = grep('Python')

# Start coroutine
next(search)

# Supply values using the .send() method
search.send("This text does not contain the keyword")
search.send("Neither does this one")
search.send("This one contains Python!")

# Close coroutine
search.close()

Searching for Python
This one contains Python!


### Singleton Pattern

*one of the most controversial patterns; many have accused it of being an "anti-pattern", a pattern that should be avoided, not promoted. In Python, if someone is using the singleton pattern, they're almost certainly doing something wrong* (Dusty Phillips, pg. 320).

#### Why Discuss It?

* commonly used in other object-oriented languages
* fundamental idea is useful to know, even if implementation in Python is different 
    * allow exactly one instance of a certain object to exist using a manager object
    * other objects can request the single instance of the manager object from the class
        * minimizes the need to pass around the reference to the manager object

In [6]:
class OneOnly:
    _singleton = None
    def __new__(cls, *args, **kwargs):
        if not cls._singleton:
            cls._singleton = super(OneOnly, cls
                ).__new__(cls, *args, **kwargs)
        return cls._singleton

In [7]:
o1 = OneOnly()
o2 = OneOnly()

o1 == o2

True

#### Why Not Use It?

* you do not know how others might want to use your code (even if you may think only one instance of the object is ever needed)
* can interfere with distributed computing, parallel programming, and automated testing, etc
    * cases where it can be very useful to have multiple or alternative instances of a specific object even if not required under typical use of object
    
#### Module-level Variables As Alternative

* provide mechanism to get access to the "default singleton" value, while also allowing creation of other instances if needed
* not technically a singleton, but provides the most Pythonic solution for singleton-like behavior

In [ ]:
# Use of module-level variable to enhance state pattern
# No wasting memory on new instances by reusing a single state object for each state

class FirstTag:
    def process(self, remaining_string, parser):
        i_start_tag = remaining_string.find('<')
        i_end_tag = remaining_string.find('>')
        tag_name = remaining_string[i_start_tag+1:i_end_tag]
        root = Node(tag_name)
        parser.root = parser.current_node = root 
        parser.state = child_node # single state object reused for this state
        return remaining_string[i_end_tag+1:]
    
class ChildNode:
    def process(self, remaining_string, parser):
        stripped = remaining_string.strip()
        
        if stripped.startswith("</"):
            parser.state = close_tag # single state object reused for this state
        elif stripped.startswith("<"):
            parser.state = open_tag # single state object reused for this state
        else:
            parser.state = text_node # single state object reused for this state
        return stripped

### Template Pattern

*designed for situations where we have several different tasks to accomplish that have some, but not all, steps in common. The common steps are implemented in a base class, and the distinct steps are overridden in subclasses to provide custom behavior* (Dusty Phillips, pg. 325) 

*the flow goes from the base class to a subclass, a principle called Inversion of Control (IoC). IoC, also called Hollywood Principle - Don’t call us, we’ll call you - decouples the execution of a task from its implementation* (<a href="https://www.giacomodebidda.com/template-method-pattern-in-python/" target="_blank">Giacomo Debidda</a>). 

#### Why Use It?

* useful for writing DRY code
* similar to strategy pattern, but selection of implementation method does not happen runtime, but rather at compile-time by subclassing the template
* capture the abstraction in an interface, and bury the implementation details in its subclasses
    * client code doesn't call implementation methods directly, but calls a template_method that then calls the implementation methods

In [13]:
# From https://github.com/jackdbd/design-patterns
import sys
from abc import ABC, abstractmethod

# Base class defining the template_method
class algorithm(ABC):

    def template_method(self):
        """Skeleton of operations to perform. DON'T override me.

        The Template Method defines a skeleton of an algorithm in an operation,
        and defers some steps to subclasses.
        """
        self.__do_absolutely_this()
        self.do_step_1()
        self.do_step_2()
        self.do_something()

    def __do_absolutely_this(self): # indicates that cannot be overridden by subclass
        """Protected operation. DON'T override me."""
        this_method_name = sys._getframe().f_code.co_name
        print('{}.{}'.format(self.__class__.__name__, this_method_name))

    @abstractmethod # indicates that must be overridden by subclass
    def do_step_1(self):
        """Primitive operation. You HAVE TO override me, I'm a placeholder."""
        pass

    @abstractmethod # indicates that must be overridden by subclass
    def do_step_2(self):
        """Primitive operation. You HAVE TO override me, I'm a placeholder."""
        pass

    def do_something(self): # can be overridden by subclass or simply used a default
        """Hook. You CAN override me, I'm NOT a placeholder."""
        print('do something')

In [14]:
# Subclass override of abstract methods
# Subclass use of default for do_something
class algorithm_a(algorithm):

    def do_step_1(self):
        print('do step 1 for Algorithm A')

    def do_step_2(self):
        print('now do step 2 for Algorithm A')

# Subclass override of abstract methods
# Subclass override of do_something
class algorithm_b(algorithm):

    def do_step_1(self):
        print('do different step 1 for Algorithm B')

    def do_step_2(self):
        print('now do different step 2 for Algorithm B')

    def do_something(self):
        print('do something else')

In [16]:
# Client code
def main():
    print('Algorithm A')
    a = algorithm_a()
    a.template_method()

    print('\nAlgorithm B')
    b = algorithm_b()
    b.template_method()

if __name__ == '__main__':
    main()

Algorithm A
algorithm_a.__do_absolutely_this
do step 1 for Algorithm A
now do step 2 for Algorithm A
do something

Algorithm B
algorithm_b.__do_absolutely_this
do different step 1 for Algorithm B
now do different step 2 for Algorithm B
do something else


## Recap

**Decorator Pattern**: useful for expanding on the response of a function/component, before/as the response is sent to a second component; allows for multiple optional behaviors (as an alternative to multiple inheritance)

**Observer Pattern**: useful for decoupling the code being observed from the code doing the observing; can have multiple observers and none to know about the others; observer also does not need to know it is being observed

**Strategy Pattern**: useful for abstracting the code doing the work from the code calling the function (i.e. Modularity; first class functions in Python); performs same task in different ways, but nothing changes for the end user/interface/client initiating the task

**State Pattern**: allow objects to switch between different states dynamically, as some process evolves; strategy/implementation method is used to choose appropriate code at runtime

**Singleton Pattern**: considered anti-pattern in Python and suggestion is to use module-level variables as an alternative

**Template Pattern**: useful for DRY code; subclasses can share a core set of functionality and have the ability to override selected methods